In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

In [2]:
class CFG(object):
    def __init__(self,
         seed: int = 42,
         n_splits: int = 5,
         show_info: bool = False,
         ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.tr_data = "../input/vehicle-loan-defaulter-tfug-delhi-challenge-1-2022/train.csv"
        self.ts_data = "../input/vehicle-loan-defaulter-tfug-delhi-challenge-1-2022/test.csv"
        self.ss_data = "../input/vehicle-loan-defaulter-tfug-delhi-challenge-1-2022/sample_submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    auc = roc_auc_score(y_pred, y_true)
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

In [4]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LogisticRegression(random_state=seed)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsClassifier()),
                ]
            )
        
        elif self.model_name == "svc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", SVC()),
                ]
            )
        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "lgbmc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LGBMClassifier(random_state=seed)),
                ]
            )

In [5]:
names: list = ["lgr", "knc", "svc", "dtc", "etc", "rfc", "gbc", "abc", "etcs", "gnb", "xgbc", "lgbmc"]
# names = names[:2]

tr_df = pd.read_csv(cfg.tr_data)
tr_df["employment_type"] = tr_df["employment_type"].map({"Salaried" : 0, "Self employed" : 1})
tr_df = tr_df.drop(columns=["unique_id"])

X = tr_df.iloc[:, :-1].copy().values
y = tr_df.iloc[:, -1].copy().values

if cfg.show_info:
    breaker()
    for val in set(y):
        print(f"Class {val} count : {y[y == val].shape[0]}")

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_auc = 0.0
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            acc, auc, pre, rec, f1 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(acc, auc, pre, rec, f1)
            print("")
        else:
            _, auc, _, _, _ = get_scores(y_valid, y_pred)

        if auc > best_auc:
            best_auc = auc
            model_fold_name = f"{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        with open(os.path.join(cfg.model_save_path, f"{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

Class False count : 31854
Class True count : 6692

**************************************************

lgr, 1

Accuracy  : 0.81621
ROC-AUC   : 0.61747
Precision : [0.98214286 0.05643994]
Recall    : [0.82657268 0.40837696]
F-Score   : [0.89766736 0.09917355]

lgr, 2

Accuracy  : 0.82306
ROC-AUC   : 0.60486
Precision : [0.97905923 0.0610687 ]
Recall    : [0.83589059 0.37383178]
F-Score   : [0.90182813 0.10498688]

lgr, 3

Accuracy  : 0.82138
ROC-AUC   : 0.61373
Precision : [0.98023839 0.06151538]
Recall    : [0.83322224 0.39423077]
F-Score   : [0.90077106 0.1064244 ]

lgr, 4

Accuracy  : 0.82203
ROC-AUC   : 0.63347
Precision : [0.97942839 0.07457122]
Recall    : [0.83404654 0.43290043]
F-Score   : [0.90091001 0.12722646]

lgr, 5

Accuracy  : 0.82138
ROC-AUC   : 0.61610
Precision : [0.97665674 0.07390649]
Recall    : [0.83543286 0.39676113]
F-Score   : [0.90054171 0.12460267]


**************************************************

knc, 1

In [6]:
ts_df = pd.read_csv(cfg.ts_data)
ts_df = ts_df.drop(columns=["unique_id"])
ts_df["employment_type"] = ts_df["employment_type"].map({"Salaried" : 0, "Self employed" : 1})
X_test = ts_df.copy().values

model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
y_pred = model.predict(X_test)

ss_df = pd.read_csv(cfg.ss_data)
ss_df["loan_defaulted"] = y_pred
ss_df.to_csv("submission.csv", index=False)

In [7]:
ss_df.head(5)

,unique_id,loan_defaulted
0,38547,False
1,38548,False
2,38549,False
3,38550,False
4,38551,False
